# 商業與大數據分析－期末專案
    會員資料: ~ 2022-04-30
    主單資料: 2013-09-30 ~ 2022-04-30
    子單資料: 2013-09-30 ~ 2022-04-30
    商品頁資料: ~ 2022-05-04
    行為資料: 2018-06 and 2021-02


In [ ]:
import pandas as pd
import numpy as np

- 會員資料

In [ ]:
# 讀取資料
member_df = pd.read_csv(r'.\91APP_MemberData.csv', encoding='utf-8')

# 將時間欄位轉成datetime格式
datecols = ['RegisterDateTime', 'Birthday', 'FirstAppOpenDateTime', 'LastAppOpenDateTime']
for col in member_df.columns:
    if col in datecols:
        member_df[col] = pd.to_datetime(member_df[col], errors='coerce')

member_df.shape

In [ ]:
member_df.info()

In [ ]:
member_df

- 主訂單

In [ ]:
# 讀取資料
order_df = pd.read_csv(r'.\91APP_OrderData.csv', encoding='utf-8')

# 將時間欄位轉成datetime格式
order_df.OrderDateTime = pd.to_datetime(order_df.OrderDateTime)

order_df.shape

In [ ]:
order_df[order_df['MemberId'] == 'PhAgr13BKhzf19pLID77a4E0/AOSWcL+2Uh8lTed5Yg=']

- 子訂單

In [ ]:
# 讀取資料
suborder_df = pd.read_csv(r'.\91APP_OrderSlaveData.csv', encoding='utf-8')

# 將時間欄位轉成datetime格式
datecols = ['OrderDateTime', 'OrderFinishDateTime']
for col in suborder_df.columns:
    if col in datecols:
        suborder_df[col] = pd.to_datetime(suborder_df[col], errors='coerce')


suborder_df.shape

- 商品頁資料

In [ ]:
# 讀取資料
salepage_df = pd.read_csv(r'.\91APP_SalePageData.csv', encoding='utf-8')

In [ ]:
salepage_df.shape

- 行為資料

In [ ]:
Behavior202102_df = pd.read_csv(r'C:\Users\Yicheng\Desktop\NTU\110-2\Bigdata Business Analysis\91APP_DataSet_2022\Behavior202102_df.csv', encoding='utf-8')

In [ ]:
# from datetime import timedelta
# Behavior202102_df.HitDateTime = Behavior202102_df.HitTime.apply(lambda x: pd.to_datetime(x, unit='ms') + timedelta(hours=8))
# Behavior202102_df.EventTime = Behavior202102_df.EventTime.apply(lambda x: pd.to_datetime(x, unit='ms') + timedelta(hours=8))

In [ ]:
Behavior202102_df.columns

In [ ]:
behavior_cols = ['Tunnel', 'FullvisitorId', 'MemberId', 'UTMSource', 'UTMMedium', 'Behavior', 'SalePageId', 'SearchTerm', 'HitDateTime', 'EventTime']
Behavior202102_df = Behavior202102_df.sort_values(['FullvisitorId', 'HitDateTime', 'EventTime'])[behavior_cols]

In [ ]:
Behavior202102_df.head(5)

In [ ]:
# 共249531個登入ID
Behavior202102_df.FullvisitorId.nunique()

In [ ]:
# 查看行為分布
Behavior202102_df.Behavior.value_counts()

In [ ]:
# 寫入檔案
# Behavior202102_df.to_csv("Behavior202102_df.csv", index=False)

# 由行為資料進行顧客分群
    找到活躍、猶豫、已購買客戶
    純訪客：註冊成為會員的轉換率
    用以下過去30天行為資料分群：
1. Session (重複瀏覽)、Session 時長(瀏覽時長)
2. 瀏覽商品頁次數
3. 搜尋次數
4. 瀏覽優惠次數(折價券詳細頁、現折活動內容頁、優惠券內容頁)
5. 加入購物車次數
6. 開始結帳次數
7. 購買次數


In [ ]:
Behavior202102_df['HitDateTime'] = pd.to_datetime(Behavior202102_df['HitDateTime'])
Behavior202102_df['EventTime'] = pd.to_datetime(Behavior202102_df['EventTime'])

In [ ]:
Behavior202102_df.info()

- 純訪客
    - MemberId為空 & 有註冊行為的ID

In [ ]:
Behavior202102_df = Behavior202102_df[(Behavior202102_df.MemberId.isna()) | (Behavior202102_df.Behavior == 'register')]

- Section 數

In [ ]:
sevtion_count = Behavior202102_df.groupby(['FullvisitorId'])['HitDateTime'].nunique()
sevtion_count = sevtion_count.reset_index()
sevtion_count

- Section平均時長

In [ ]:
section_time = Behavior202102_df.groupby(['FullvisitorId', 'HitDateTime'])['EventTime'].max() - Behavior202102_df.groupby(['FullvisitorId', 'HitDateTime'])['EventTime'].min()
section_avgtime = section_time.reset_index().groupby('FullvisitorId')['EventTime'].mean()
section_avgtime

- 瀏覽商品頁次數

In [ ]:
viewproduct_times = Behavior202102_df[Behavior202102_df['Behavior'] == 'viewproduct'].groupby(['FullvisitorId'])['Behavior'].value_counts()
viewproduct_times.index = viewproduct_times.index.droplevel(1)
viewproduct_times = viewproduct_times.reset_index()
viewproduct_times.rename(columns={'Behavior' : 'viewproduct'}, inplace=True)
viewproduct_times

- 搜尋次數

In [ ]:
search_times = Behavior202102_df[Behavior202102_df['Behavior'] == 'search'].groupby('FullvisitorId')['Behavior'].count().reset_index()
search_times.rename(columns={'Behavior': 'search'}, inplace=True)
search_times

- 優惠瀏覽次數

In [ ]:
coupon_times = Behavior202102_df[(Behavior202102_df['Behavior'] == 'viewecoupondetail') | (Behavior202102_df['Behavior'] == 'viewpromotiondetail') | (Behavior202102_df['Behavior'] == 'viewcoupondetail')].groupby(['FullvisitorId'])['Behavior'].count().reset_index()
coupon_times.rename(columns={'Behavior': 'viewcoupon'}, inplace=True)
coupon_times

- 加入購物車次數

In [ ]:
add_times = Behavior202102_df[Behavior202102_df['Behavior'] == 'add'].groupby('FullvisitorId')['Behavior'].count().reset_index()
add_times.rename(columns={'Behavior': 'add'}, inplace=True)
add_times

- 開始結帳次數

In [ ]:
checkout_times = Behavior202102_df[Behavior202102_df['Behavior'] == 'checkout'].groupby('FullvisitorId')['Behavior'].count().reset_index()
checkout_times.rename(columns={'Behavior': 'checkout'}, inplace=True)
checkout_times

- 購買次數

In [ ]:
purchase_times = Behavior202102_df[Behavior202102_df['Behavior'] == 'purchase'].groupby('FullvisitorId')['Behavior'].count().reset_index()
purchase_times.rename(columns={'Behavior': 'purchase'}, inplace=True)
purchase_times

# 彙整資料框

In [ ]:
df = sevtion_count.join(section_avgtime, on='FullvisitorId')
df = df.merge(viewproduct_times, on='FullvisitorId', how='left')
df = df.merge(coupon_times, on='FullvisitorId', how='left')
df = df.merge(add_times, on='FullvisitorId', how='left')
df = df.merge(purchase_times, on='FullvisitorId', how='left')
df = df.merge(checkout_times, on='FullvisitorId', how='left')
df = df.merge(search_times, on='FullvisitorId', how='left')

df.rename(columns={'HitDateTime': 'section_count', 'EventTime': 'section_time'}, inplace=True)

# 將timedelta64轉成以秒為單位的int
df.section_time = (df.section_time / np.timedelta64(1,'s')).astype(int)
df.info()

In [ ]:
# 以0填充na
df = df.fillna(0)

In [ ]:
# 移除section_count異常
df = df[df.section_count < 200]
# 移除section_time異常
df = df[df.section_time < 10000]


# K-means分群

In [ ]:
k_means_df = df.iloc[:,1:]

In [ ]:
from sklearn.cluster import KMeans
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=17)
    kmeans.fit(k_means_df)
    sse[k] = kmeans.inertia_


In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
plt.title('The Elbow Method')
plt.xlabel('k'); plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

In [ ]:
# 將30天內有註冊成會員的ID篩選出來
registor = Behavior202102_df[Behavior202102_df.Behavior == 'register'][['FullvisitorId']]
registor['register'] = 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#正規化
Min_Max_Scaler = MinMaxScaler( feature_range=(0,1) ) # 設定縮放的區間上下限
MinMax_Data = Min_Max_Scaler.fit_transform( k_means_df ) # Data 為原始資料

#分群中心
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=0)
kmeans_fit = kmeans.fit_predict(MinMax_Data)
print(round(pd.DataFrame(kmeans.cluster_centers_),4))
print(kmeans.inertia_)
df2 = pd.DataFrame(kmeans_fit, columns=['label'])
df['label'] = df2

# MAP到分群結果
df = df.merge(registor, on='FullvisitorId', how='left')
df = df.fillna(0)
df.rename(columns={'register_y':'register'}, inplace=True)

#各分群的註冊率
for i in range(3):
    num_regiser=df[(df.label == i)]['register'].sum()
    num_label=df[(df.label == i)]['FullvisitorId'].count()
    print(num_regiser,num_label,num_regiser/num_label)
#各分群的購買率
print('------------------------------------------')
for i in range(3):
    num_purchas=df[(df.label == i)]['purchase'].sum()
    num_label=df[(df.label == i)]['FullvisitorId'].count()
    print(num_purchas,num_label,num_purchas/num_label)

In [ ]:
df.label.value_counts()

- 分群特徵平均
    - 0: 非活躍訪客
    - 1: 機會訪客
    - 2: 活躍訪客

In [ ]:
df.groupby(['label']).agg({'section_count': 'mean', 'section_time': 'mean', 'viewproduct': 'mean', 'viewcoupon': 'mean', 'add': 'mean', 'purchase': 'mean', 'checkout': 'mean', 'search': 'mean'})

- 相對重要性

In [ ]:
cluster_avg = df.groupby(['Cluster']).mean()
population_avg = df.mean()
relative_imp = cluster_avg / population_avg - 1

In [ ]:
relative_imp.round(2)

In [ ]:
# Plot heatmap
plt.figure(figsize=(8, 2))
plt.title('Relative importance of attributes')
sns.heatmap(data=relative_imp, annot=True, fmt='.2f', cmap='RdYlGn')
plt.show()

# 純訪客的各分群成效
    會員註冊率

- 註冊會員

In [ ]:
# 分群成效
register_cluster = df.groupby('label').register.value_counts().unstack()
register_cluster
register_cluster['registor_rate'] = register_cluster[1] / register_cluster[0]
register_cluster

# 201806的整理、分析方法大同小異，附在另一個檔案上